In [1]:
import numpy as np
import pandas as pd

In [2]:
print("This is the answer to part 1 of the exercise\n")

train_ds_comma=pd.read_csv("train.csv")
# train_ds_line=pd.read_csv("train.csv",sep='|')


This is the answer to part 1 of the exercise



In [5]:
print(train_ds_comma[0:10])
total_rows=len(train_ds_comma)
print((int) (total_rows*0.1)) # ten percent of the data  set
# print(train_ds_line)

       Id                                              Title  \
0  227464  Netflix is coming to cable boxes, and Amazon i...   
1  244074  Pharrell, Iranian President React to Tehran 'H...   
2   60707                    Wildlife service seeks comments   
3   27883  Facebook teams up with Storyful to launch 'FB ...   
4  169596           Caesars plans US$880 mln New York casino   
5   48507  TweetDeck Briefly Shuts Down in Response to Se...   
6  220838          OMG: Miley Cyrus Postpones 'Bangerz' Tour   
7   56638  Have a Nexus 5 or new Nexus 7? You can get And...   
8  282792  Legendary blues guitarist Johnny Winter dies i...   
9  288645  Kardashian Siblings Slammed by Critics for Tex...   

                                             Content          Label  
0   if you subscribe to one of three rinky-dink (...  Entertainment  
1   pharrell, iranian president react to tehran '...  Entertainment  
2   the u.s. fish and wildlife service has reopen...     Technology  
3   the very na